In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

import pandas as pd
import numpy as np
import keras
from keras import backend as K

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt

In [2]:
if tf.test.is_gpu_available():
    print("GPU is available.")
else:
    print("GPU is NOT available.")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available.


In [ ]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# # Original: EfficientNetB0 feature vector (version 1)
# efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [3]:
df = pd.read_csv("C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_2019_Training_Metadata.csv").drop("lesion_id", axis=1)
im_df = pd.read_csv("C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_2019_Training_GroundTruth.csv").drop("UNK", axis=1)
im_df["labels"] = im_df.iloc[:, 1:].idxmax(axis=1)

df = df.join(im_df.set_index("image"), on=["image"]).drop(["MEL", "NV", "BCC", "AK", "DF", "VASC", "SCC", "BKL"], axis=1)
df = df[~df["image"].str.contains("downsampled")]

skin_lesion_dict = {
    "MEL": "Melanoma",
    "NV": "Melanocytic Nevus",
    "BCC": "Basal Cell Carcinoma",
    "AK": "Actinic Keratosis",
    "BKL": "Benign Keratosis",
    "DF": "Dermatofibroma",
    "VASC": "Vascular Lesion",
    "SCC": "Squamous Cell Carcinoma"
}

df['cell_type'] = df['labels'].map(skin_lesion_dict.get) 

# Define the directory where the images are located
image_directory = "C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_2019_Training_Input/"

# Create a new column 'image_path' in the DataFrame
df['path'] = df['image'].apply(lambda filename: os.path.join(image_directory, filename))


df['cell_type_idx'] = pd.Categorical(df['cell_type']).codes
# labels = {"NV": 0, "MEL": 1, "BCC": 1, "BKL": 0, "AK": 0, "SCC": 1, "VASC": 0, "DF": 0}


df.head()

,image,age_approx,anatom_site_general,sex,labels,cell_type,path,cell_type_idx
0,ISIC_0000000,55.0,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4
1,ISIC_0000001,30.0,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4
2,ISIC_0000002,60.0,upper extremity,female,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5
3,ISIC_0000003,30.0,upper extremity,male,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4
4,ISIC_0000004,80.0,posterior torso,male,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5


In [4]:
# Assuming you already have the 'cell_type_idx' column in your DataFrame 'df'
# Group the DataFrame by 'cell_type' and iterate through the groups
grouped = df.groupby(['cell_type', 'cell_type_idx'])

for name, group in grouped:
    print(f"Value: {name[0]}, Code: {name[1]}")


Value: Actinic Keratosis, Code: 0
Value: Basal Cell Carcinoma, Code: 1
Value: Benign Keratosis, Code: 2
Value: Dermatofibroma, Code: 3
Value: Melanocytic Nevus, Code: 4
Value: Melanoma, Code: 5
Value: Squamous Cell Carcinoma, Code: 6
Value: Vascular Lesion, Code: 7


In [ ]:
# df['path'] = df['path'] + '.jpg'

# # Load images, convert to NumPy arrays, and add them to a new 'image' column
# df['image'] = df['path'].map(lambda x: np.asarray(Image.open(x)))

# # Sample images
# n_samples = 5
# fig, m_axs = plt.subplots(8, n_samples, figsize=(4 * n_samples, 3 * 8))
# for n_axs, (type_name, type_rows) in zip(m_axs, df.sort_values(['cell_type']).groupby('cell_type')):
#     n_axs[0].set_title(type_name)
#     for c_ax, (_, c_row) in zip(n_axs, type_rows.sample(n_samples, random_state=1234).iterrows()):
#         c_ax.imshow(c_row['image'])
#         c_ax.axis('off')
# fig.savefig('category_samples.png', dpi=500)
# plt.show()


In [5]:
df.isnull().sum()

image                     0
age_approx              223
anatom_site_general    1883
sex                     204
labels                    0
cell_type                 0
path                      0
cell_type_idx             0
dtype: int64

In [6]:
df["age_approx"] = df["age_approx"].fillna(df["age_approx"].mean()).astype("int")
df["anatom_site_general"] = df["anatom_site_general"].fillna("unknown")
df["sex"] = df["sex"].fillna("unknown")

df["age_bins"] = pd.cut(df["age_approx"], bins=[-1, 20, 40, 60, 80, 95], labels=False)
df.head()

,image,age_approx,anatom_site_general,sex,labels,cell_type,path,cell_type_idx,age_bins
0,ISIC_0000000,55,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,2
1,ISIC_0000001,30,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,1
2,ISIC_0000002,60,upper extremity,female,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,2
3,ISIC_0000003,30,upper extremity,male,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,1
4,ISIC_0000004,80,posterior torso,male,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,3


In [7]:
df.isnull().sum()

image                  0
age_approx             0
anatom_site_general    0
sex                    0
labels                 0
cell_type              0
path                   0
cell_type_idx          0
age_bins               0
dtype: int64

In [8]:
df["labels"].value_counts(normalize=True)

labels
NV      0.497012
MEL     0.178355
BCC     0.142882
BKL     0.096315
AK      0.037279
SCC     0.027003
VASC    0.010878
DF      0.010276
Name: proportion, dtype: float64

In [9]:
df.head()

,image,age_approx,anatom_site_general,sex,labels,cell_type,path,cell_type_idx,age_bins
0,ISIC_0000000,55,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,2
1,ISIC_0000001,30,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,1
2,ISIC_0000002,60,upper extremity,female,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,2
3,ISIC_0000003,30,upper extremity,male,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,1
4,ISIC_0000004,80,posterior torso,male,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,3


In [ ]:
# df.value_counts

In [10]:
df_ = df.groupby('image').count()
df_.head()

,age_approx,anatom_site_general,sex,labels,cell_type,path,cell_type_idx,age_bins
image,,,,,,,,
ISIC_0000000,1,1,1,1,1,1,1,1
ISIC_0000001,1,1,1,1,1,1,1,1
ISIC_0000002,1,1,1,1,1,1,1,1
ISIC_0000003,1,1,1,1,1,1,1,1
ISIC_0000004,1,1,1,1,1,1,1,1


In [11]:
df_.reset_index(inplace=True)

In [12]:
df_.head()

,image,age_approx,anatom_site_general,sex,labels,cell_type,path,cell_type_idx,age_bins
0,ISIC_0000000,1,1,1,1,1,1,1,1
1,ISIC_0000001,1,1,1,1,1,1,1,1
2,ISIC_0000002,1,1,1,1,1,1,1,1
3,ISIC_0000003,1,1,1,1,1,1,1,1
4,ISIC_0000004,1,1,1,1,1,1,1,1


In [13]:
df_.value_counts()

image         age_approx  anatom_site_general  sex  labels  cell_type  path  cell_type_idx  age_bins
ISIC_0000000  1           1                    1    1       1          1     1              1           1
ISIC_0060937  1           1                    1    1       1          1     1              1           1
ISIC_0060935  1           1                    1    1       1          1     1              1           1
ISIC_0060934  1           1                    1    1       1          1     1              1           1
ISIC_0060933  1           1                    1    1       1          1     1              1           1
                                                                                                       ..
ISIC_0031228  1           1                    1    1       1          1     1              1           1
ISIC_0031227  1           1                    1    1       1          1     1              1           1
ISIC_0031226  1           1                    1   

In [14]:
# here we identify lesion_id's that have duplicate images and those that have only
# one image.

def identify_duplicates(x):
    
    unique_list = list(df_['image'])
    
    if x in unique_list:
        return 'no_duplicates'
    else:
        return 'has_duplicates'
    
# create a new colum that is a copy of the lesion_id column
df['duplicates'] = df['image']
# apply the function to this new column
df['duplicates'] = df['duplicates'].apply(identify_duplicates)

df.head()

,image,age_approx,anatom_site_general,sex,labels,cell_type,path,cell_type_idx,age_bins,duplicates
0,ISIC_0000000,55,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,2,no_duplicates
1,ISIC_0000001,30,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,1,no_duplicates
2,ISIC_0000002,60,upper extremity,female,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,2,no_duplicates
3,ISIC_0000003,30,upper extremity,male,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,1,no_duplicates
4,ISIC_0000004,80,posterior torso,male,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,3,no_duplicates


In [15]:
df['duplicates'].value_counts()

duplicates
no_duplicates    23257
Name: count, dtype: int64

In [16]:
# now we filter out images that don't have duplicates
df_ = df[df['duplicates'] == 'no_duplicates']

df_.shape

(23257, 10)

In [17]:
df.head()
df.columns

Index(['image', 'age_approx', 'anatom_site_general', 'sex', 'labels',
       'cell_type', 'path', 'cell_type_idx', 'age_bins', 'duplicates'],
      dtype='object')

In [18]:
df_.head()

,image,age_approx,anatom_site_general,sex,labels,cell_type,path,cell_type_idx,age_bins,duplicates
0,ISIC_0000000,55,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,2,no_duplicates
1,ISIC_0000001,30,anterior torso,female,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,1,no_duplicates
2,ISIC_0000002,60,upper extremity,female,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,2,no_duplicates
3,ISIC_0000003,30,upper extremity,male,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,1,no_duplicates
4,ISIC_0000004,80,posterior torso,male,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,3,no_duplicates


In [19]:
y = df_['labels']
y.value_counts()


labels
NV      11559
MEL      4148
BCC      3323
BKL      2240
AK        867
SCC       628
VASC      253
DF        239
Name: count, dtype: int64

In [20]:
_, df_val = train_test_split(df_, test_size=0.2, random_state=101, stratify=y)

df_val.shape

(4652, 10)

In [21]:
df_val.head()

,image,age_approx,anatom_site_general,sex,labels,cell_type,path,cell_type_idx,age_bins,duplicates
12135,ISIC_0033538,70,lower extremity,male,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,3,no_duplicates
7886,ISIC_0029289,60,unknown,female,BKL,Benign Keratosis,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,2,2,no_duplicates
3544,ISIC_0024947,40,head/neck,male,BKL,Benign Keratosis,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,2,1,no_duplicates
3401,ISIC_0024804,55,anterior torso,male,NV,Melanocytic Nevus,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,4,2,no_duplicates
23859,ISIC_0070927,85,lower extremity,female,MEL,Melanoma,C:/Users/cl502_12/Desktop/ISIC_Dataset/ISIC_20...,5,4,no_duplicates


In [22]:
df_val['labels'].value_counts()

labels
NV      2312
MEL      830
BCC      665
BKL      448
AK       173
SCC      126
VASC      50
DF        48
Name: count, dtype: int64

In [23]:
# This set will be df_data excluding all rows that are in the val set

# This function identifies if an image is part of the train
# or val set.
def identify_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image'])
    
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# identify train and val rows

# create a new colum that is a copy of the image_id column
df['train_or_val'] = df['image']
# apply the function to this new column
df['train_or_val'] = df['train_or_val'].apply(identify_val_rows)
   
# filter out train rows
df_train = df[df['train_or_val'] == 'train']


print(len(df_train))
print(len(df_val))

18605
4652


In [24]:
df_train['labels'].value_counts()

labels
NV      9247
MEL     3318
BCC     2658
BKL     1792
AK       694
SCC      502
VASC     203
DF       191
Name: count, dtype: int64

In [25]:
df_val['labels'].value_counts()

labels
NV      2312
MEL      830
BCC      665
BKL      448
AK       173
SCC      126
VASC      50
DF        48
Name: count, dtype: int64

In [26]:
base_dir = 'C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/'
train_dir = 'C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/train/' 
val_dir = 'C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/val/'

In [ ]:
# # Copy the train images into aug_dir
# class_list = ["MEL",
#     "NV",
#     "BCC",
#     "AK",
#     "BKL",
#     "DF",
#     "VASC",
#     "SCC"]

#  # We are creating temporary directories here because we delete these directories later
#     # create a base dir


# for item in class_list:
    
#     aug_dir = 'C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/aug_dir'
#     os.mkdir(aug_dir)
#     # create a dir within the base dir to store images of the same class
#     img_dir = os.path.join(aug_dir, 'img_dir')
#     os.mkdir(img_dir)

#     # Choose a class
#     img_class = item

#     # list all images in that directory
#     img_list = os.listdir('C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/train/' + img_class)

#     # Copy images from the class train dir to the img_dir e.g. class 'mel'
#     for fname in img_list:
#             # source path to image
#             src = os.path.join('C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/train/' + img_class, fname)
#             # destination path to image
#             dst = os.path.join(img_dir, fname)
#             # copy the image from the source to the destination
#             shutil.copyfile(src, dst)


#     # point to a dir containing the images and not to the images themselves
#     path = aug_dir
#     save_path = 'C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/train/' + img_class

#     # Create a data generator
#     datagen = ImageDataGenerator(
#         rotation_range=180,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         zoom_range=0.1,
#         horizontal_flip=True,
#         vertical_flip=True,
#         brightness_range=(0.9,1.1),
#         fill_mode='nearest')

#     batch_size = 50

#     aug_datagen = datagen.flow_from_directory(path,
#                                            save_to_dir=save_path,
#                                            save_format='jpg',
#                                                     target_size=(224,224),
#                                                     batch_size=batch_size)



#     # Generate the augmented images and add them to the training folders
    
#     ###########
    
#     num_aug_images_wanted = 6000 # total number of images we want to have in each class
    
#     ###########
    
#     num_files = len(os.listdir(img_dir))
#     num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

#     # run the generator and create about 6000 augmented images
#     for i in range(0,num_batches):

#         imgs, labels = next(aug_datagen)
        
#     # delete temporary directory with the raw image files
#     shutil.rmtree('C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/aug_dir')

In [ ]:
# def plots(ims, figsize=(12,6), rows=5, interp=False, titles=None): # 12,6
#     if type(ims[0]) is np.ndarray:
#         ims = np.array(ims).astype(np.uint8)
#         if (ims.shape[-1] != 3):
#             ims = ims.transpose((0,2,3,1))
#     f = plt.figure(figsize=figsize)
#     cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
#     for i in range(len(ims)):
#         sp = f.add_subplot(rows, cols, i+1)
#         sp.axis('Off')
#         if titles is not None:
#             sp.set_title(titles[i], fontsize=16)
#         plt.imshow(ims[i], interpolation=None if interp else 'none')
        
# plots(imgs, titles=None) # titles=labels will display the image labels

In [27]:
base_dir = 'C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/'
train_dir = 'C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/train/' 
val_dir = 'C:/Users/cl502_12/Desktop/CUSTOM_DATASET_ISIC_2019/val/'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 32
val_batch_size = 32
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [28]:
print(num_train_samples) 
print(num_val_samples) 


18605
4652


In [29]:
datagen = ImageDataGenerator(
    preprocessing_function= \
    tf.keras.applications.resnet_v2.preprocess_input)

train_batches = datagen.flow_from_directory(train_dir,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size)

valid_batches = datagen.flow_from_directory(val_dir,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size)

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = datagen.flow_from_directory(val_dir,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)

Found 54372 images belonging to 8 classes.
Found 9174 images belonging to 8 classes.
Found 9174 images belonging to 8 classes.


In [30]:
from vit_keras import vit
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import math

c:\Users\cl502_12\Desktop\FINAL_YEAR_PROJEC\.venv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\cl502_12\Desktop\FINAL_YEAR_PROJEC\.venv\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug,

In [33]:
vit_model = vit.vit_b16(
        image_size = 224,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 8)

In [34]:
model1 = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(11, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(8, 'softmax')
    ],
    name = 'vision_transformer')

model1.summary()

Model: "vision_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b16 (Functional)        (None, 768)               85798656  
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 batch_normalization (BatchN  (None, 768)              3072      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 11)                8459      
                                                                 
 batch_normalization_1 (Batc  (None, 11)               44        
 hNormalization)                                

In [42]:
model1.compile(loss="categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                        optimizer=tf.keras.optimizers.Adam(0.001), # 10x lower learning rate than the default
                        metrics=["accuracy"])

In [43]:
early_stopping = EarlyStopping(monitor='val_loss',patience=2)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = 'model2.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1)

In [136]:
# # early_stopping = EarlyStopping(monitor='val_loss',patience=5)

# checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = 'ISIC_model.hdf5',
#                                                   monitor = 'val_accuracy', 
#                                                   verbose = 1, 
#                                                   save_best_only = True,
#                                                   save_weights_only = True,
#                                                   mode = 'max')

# # def step_decay(epoch):
# #     initial_lrate = 0.0001
# #     drop = 0.1
# #     epochs_drop = 10.0
# #     lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
# #     return lrate
# # lr_scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1)


# # Define a custom learning rate schedule with step decay
# def step_decay_schedule(initial_lr, decay_factor, step_size):
#     def schedule(epoch, lr):
#         if (epoch + 1) % step_size == 0:
#             return lr * decay_factor
#         return lr

#     return schedule


# # Learning rate schedule with step decay
# lr_schedule = tf.keras.callbacks.LearningRateScheduler(
#     schedule=step_decay_schedule(initial_lr=0.001, decay_factor=0.5, step_size=5)
# )


# # Creating learning rate reduction callback
# lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
#                                                  factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
#                                                  patience=5,
#                                                  verbose=1, # print out when learning rate goes down 
#                                                  min_lr=1e-7)

In [44]:
# callbacks = [early_stopping, checkpointer, lr_scheduler]
callbacks = [checkpointer, early_stopping, lr_scheduler]

In [45]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [46]:
history = model1.fit(train_batches, steps_per_epoch=train_steps,
                              validation_data=valid_batches,
                              validation_steps=val_steps,
                              epochs=40,
                              callbacks=callbacks)


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/40


KeyboardInterrupt: 

In [147]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

In [47]:
# plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
# plt.title('Model Accuracy: Training and Validation')
# plt.legend(loc=0)
# plt.figure()

In [48]:
# plt.plot(epochs, loss, 'r', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation Loss')
# plt.legend(loc=0)
# plt.figure()

In [150]:
from tensorflow.keras.models import Model
from keras import models

In [151]:
model_test_performace = model1.evaluate(test_batches)

  36/4976 [..............................] - ETA: 10:29 - loss: 2.3576 - accuracy: 0.3333

KeyboardInterrupt: 

In [ ]:
test_accuracy = model_test_performace[1]

print(f'Test accuracy: {round(test_accuracy*100, 2)}%')

In [152]:
test_labels = test_batches.classes

In [153]:
test_batches.class_indices

{'AK': 0, 'BCC': 1, 'BKL': 2, 'DF': 3, 'MEL': 4, 'NV': 5, 'SCC': 6, 'VASC': 7}

In [154]:
test_labels.shape

(4976,)

In [155]:
predictions = model.predict_generator(test_batches, verbose=1)

C:\Users\cl502_12\AppData\Local\Temp\ipykernel_33516\2830036314.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_batches, verbose=1)


  61/4976 [..............................] - ETA: 11:47

KeyboardInterrupt: 

In [ ]:
predictions.shape

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
test_labels.shape

In [ ]:
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = ['AK' ,'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')